In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import copy  ## for deepcopy

So a metaKG edge has:
* one subject and one object (one way, both are BioThings types/Biolink Model entities, they can be the same type)
* one predicate
* one "source"/path of provenance

Note that this API is built from the tab-delimited download files from DisGeNET (not the SQLite, RDF, or Cytoscape files). 

Note: talked to Kevin about using v7.0 for the version. 

**note to self: the parsing of this response below to get the other properties out, build publications/website fields will have go after getting this back and perhaps before the mapping/augmentation done on the response below**

In [2]:
## example of JSON response for genes_related_to_disease:
sample_GDA_response = {  
        "DPI": 0.846,     ## map to response with augmentation
        "DSI": 0.536,     ## map to response with augmentation
        "EI": 0.917,      ## map to response with augmentation
        "YearFinal": 2019,  ## CURRENTLY SKIPPED/NOT MAPPED
        "YearInitial": 1998,  ## CURRENTLY SKIPPED/NOT MAPPED
        "gene_id": 9,  ## mapped to response (output)
        "gene_name": "NAT1",
        "pubmed": [9610785,10090301,10698485,12835615,12860276,   ## map to response with augmentation
                   14517345,15084249,15090724,15226672,16049806,
                   17010218,17973251,18288399,22333393,24467436,
                   25528056,27648926,28359264,29315819,29339455,
                   29901116,29964355,29969986,31358821],
        "score": 0.1,  ## map to response with augmentation
        ## this field is replaced 
        ## "source": "BEFREE"
      }

## x-bte-reusable-info

In [3]:
x_bte_reusable_info = {
    ## put here since it's used repeatedly
    "request": {
        ## BELOW DOESN'T WORK
        ## API MAY NEED CHANGES TO "FILTER"/"SEARCH" DOCUMENTS RETURNED AND SET SOURCE FIELD AS A SPECIFIC FIELD 
        "disgenet-LHGDN_input-disease": {
            "body": {
                "q": '{inputs[0]&source=LHGDN}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},
        "disgenet-BEFREE_input-disease": {
            "body": {
                "q": '{inputs[0]&source=BEFREE}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"}, 
        "disgenet-HPO_input-disease": {
            "body": {
                "q": '{inputs[0]&source=HPO}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},     
        "disgenet-UNIPROT_input-disease": {
            "body": {
                "q": '{inputs[0]&source=UNIPROT}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},  
        "disgenet-CGI_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CGI}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},  
        "disgenet-CLINVAR_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CLINVAR}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},   
        "disgenet-GWASCAT_input-disease": {
            "body": {
                "q": '{inputs[0]&source=GWASCAT}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},        
        "disgenet-GWASDB_input-disease": {
            "body": {
                "q": '{inputs[0]&source=GWASDB}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},        
        "disgenet-MGD_input-disease": {
            "body": {
                "q": '{inputs[0]&source=MGD}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},  
        "disgenet-RGD_input-disease": {
            "body": {
                "q": '{inputs[0]&source=RGD}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},     
        "disgenet-CTDmouse_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CTD_mouse}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},        
        "disgenet-CTDrat_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CTD_rat}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},    
        "disgenet-CLINGEN_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CLINGEN}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},      
        "disgenet-GENOMICS-ENG_input-disease": {
            "body": {
                "q": '{inputs[0]&source=GENOMICS_ENGLAND}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},      
        "disgenet-CTDhuman_input-disease": {
            "body": {
                "q": '{inputs[0]&source=CTD_human}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},   
        "disgenet-PSYGENET_input-disease": {
            "body": {
                "q": '{inputs[0]&source=PSYGENET}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"}, 
        "disgenet-ORPHANET_input-disease": {
            "body": {
                "q": '{inputs[0]&source=ORPHANET}',
                "scopes": 'mondo.xrefs.umls, disgenet.xrefs.umls'},
            "header": "application/x-www-form-urlencoded"},            
    }, 
    "input-disease_UMLS":[
        {"id": "UMLS", "semantic": "Disease"}
    ],
    "output-gene_disgenet_NCBIGene":[
        ## replaced x-bte-response-mapping with this and 
        ##   other mappings to response fields for publications and measure values   
        {"id": "NCBIGene", "semantic": "Gene", 
         "mapped_response_field": sample_GDA_response["gene_id"]}
    ],      
    "predicate_is-associated_with":
        ## biolink maps to TRAPI/biolink-model predicate
        ## id maps to TRAPI/biolink-model relation
        ## biolink is required, id and label are optional, id must be a CURIE   
        {
            "biolink": "related_to",
            "id": 'SIO:001403',
            "label": "is_associated_with"            
        },
    "provenance":
    {
        "mydisease-method-ingest-consolidate": {
            "name": "MyDisease.info API",
            "knowledge_source_type": "service",
            ## trying to get these dates parsed as simple strings
            "version": "2020-10-26",
            "method": "ingest_consolidate"              
        },
        "mydisease-method-ingest-only": {
            "name": "MyDisease.info API",
            "knowledge_source_type": "service",
            ## trying to get these dates parsed as simple strings
            "version": "2020-10-26",
            "method": "ingest"               
        },
        "disgenet-method-NLP_LHGDN": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "NLP_LHGDN",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },
        "disgenet-LHGDN_origin": {
            "name": "GeneRIF",
            "knowledge_source_type": "text",
            ## trying to get these dates parsed as simple strings          
            "version": "2009-03-31",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:9606']
            }
        },
        "disgenet-method-NLP_BEFREE": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "NLP_BEFREE",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },        
        "disgenet-BEFREE_origin": {
            "name": "MEDLINE_abstracts",
            "knowledge_source_type": "publications",
            "version": "1970-01_to_2019-12"
        },
        "disgenet-method-propagate-phenotype": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "propagate_from_phenotype",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },  
        "disgenet-HPO_origin": {
            "name": "HPO_annotations",
            "knowledge_source_type": "knowledgebase"
        },  
        "disgenet-method-propagate-protein-var": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "propagate_from_protein_variant",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },  
        "disgenet-UNIPROT_origin": {
           "name": "UniProt",
            "knowledge_source_type": "knowledgebase",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:9606']
            }
        },
        "disgenet-method-ingest-only": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "ingest",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },  
        "disgenet-CGI_origin": {
           "name": "Cancer_Genome_Interpreter",
            "knowledge_source_type": "knowledgebase",
        },      
        "disgenet-method-propagate-seq-var": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "propagate_from_sequence_variant",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },  
        "disgenet-CLINVAR_origin": {
           "name": "ClinVar",
            "knowledge_source_type": "knowledgebase",
        },    
        "disgenet-GWASCAT_origin": {
           "name": "NHGRI_EBI_GWAS_CATALOG",
            "knowledge_source_type": "knowledgebase",
        },       
        "disgenet-GWASDB_origin": {
           "name": "GWASdb",
            "knowledge_source_type": "knowledgebase",
        },   
        "disgenet-method-propagate-orthology": {
            "name": "DisGeNET",
            "knowledge_source_type": "knowledgebase",
            "version": "v7.0",
            "method": "propagate_from_orthology",
            "method_info": {
                "url": ['https://www.disgenet.org/dbinfo#section11']
            }
        },  
        "disgenet-MGD_origin": {
           "name": "MGD",
            "knowledge_source_type": "knowledgebase",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:10090']
            }
        },  
        "disgenet-RGD_origin": {
            "name": "RGD",
            "knowledge_source_type": "knowledgebase"
        },  
        "disgenet-CTDmouse_origin": {
           "name": "CTD",
            "knowledge_source_type": "knowledgebase",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:10090']
            }
        },   
        "disgenet-CTDrat_origin": {
           "name": "CTD",
            "knowledge_source_type": "knowledgebase",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:10116']
            }
        },  
        "disgenet-CLINGEN_origin": {
           "name": "ClinGen",
            "knowledge_source_type": "knowledgebase",
        },   
        "disgenet-GENOMICS-ENG_origin": {
           "name": "Genomics_England_PanelApp",
            "knowledge_source_type": "knowledgebase",
        },   
        "disgenet-CTDhuman_origin": {
           "name": "CTD",
            "knowledge_source_type": "knowledgebase",
            "knowledge_source_context": {
                "taxon_specific": ['NCBITaxon:9606']
            }
        },  
        "disgenet-PSYGENET_origin": {
           "name": "PsyGeNET",
            "knowledge_source_type": "knowledgebase",
        },  
        "disgenet-ORPHANET_origin": {
           "name": "Orphanet",
            "knowledge_source_type": "knowledgebase",
        },          
    },
    ## all disease-gene and gene-disease operations using disgenet will have these properties 
    "disgenet_dggd": {  
        ## the rest is specific to DisGeNET's current gene-disease association TSV data dump
        "nodes_conflated":[
            {"original_type": "Disease",
             "conflated_type": "PhenotypicFeature",
             "where": "DisGeNET"},
            {"original_type": "Gene",
             "conflated_type": "GeneProduct",
             "where": "DisGeNET"}],          
        "numeric_measures": 
        [
            {"name":"GDAscore",
             "standard_label":"association_score",  ## name Translator may want to use
             "value": sample_GDA_response["score"],  ## mapped to response field 
             "range":{"minExclusive":0,"maxInclusive":1},
             "numeric_direction_info":
                 ## score is a measure of "how much evidence there is", so it can be related to confidence 
                 {"larger": "more_evidence"},
             "measure_info":
                 {"url":["https://www.disgenet.org/dbinfo#section31"]}
            },
            {"name":"EI",
             "standard_label":"evidence_index",  
             "value": sample_GDA_response["EI"],
             "range":{"minExclusive":0,"maxInclusive":1},
             "numeric_direction_info":
                 ## score is a measure of "how often a publication says there is vs there isn't an association" 
                 {"larger": "more_consistent"},
             "measure_info":
                 {"url":["https://www.disgenet.org/dbinfo#section36"]}
            },
            {"name":"DSI",
             "standard_label":"gene_specific_to_disease",  ## name Translator may want to use
             "value": sample_GDA_response["DSI"],
             "ontology_term": "SIO:001351",  ## SIO has a class for this
             "range":{"minExclusive":0,"maxInclusive":1},
             "numeric_direction_info":
                 {"more_specific": "larger"},
             "measure_info":
                 {"url":["https://www.disgenet.org/dbinfo#section33"]}
            },
            {"name":"DPI",
             "standard_label":"gene_specific_to_disease_class",  ## name Translator may want to use
             "value": sample_GDA_response["DPI"],
             "ontology_term": "SIO:001352",  ## SIO has a class for this
             "range":{"minExclusive":0,"maxInclusive":1},
             "numeric_direction_info":
                 {"more_specific": "smaller"},
             "measure_info":
                 {"url":["https://www.disgenet.org/dbinfo#section34"]}
            },
        ],                  
    }
}

## Disease -> Gene 

### For LHGDN

In [4]:
disease_gene1_disgenet_lhgdn = {
    ## the modified old metadata for the query
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-LHGDN_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    ## new metadata stuff starts here
    ## PREDICATE-RELATED: WHAT IS THE RELATIONSHIP
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    ## PROVENANCE-RELATED/WHERE THE RELATIONSHIP CAME FROM
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-NLP_LHGDN"],
        x_bte_reusable_info["provenance"]["disgenet-LHGDN_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    ## MEASURES
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [5]:
disease_gene1_disgenet_lhgdn["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'NLP_LHGDN',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'GeneRIF',
  'knowledge_source_type': 'text',
  'version': '2009-03-31',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:9606']}}]

### For BEFREE

In [6]:
disease_gene2_disgenet_befree = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-BEFREE_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-NLP_BEFREE"],
        x_bte_reusable_info["provenance"]["disgenet-BEFREE_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [7]:
disease_gene2_disgenet_befree["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'NLP_BEFREE',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'MEDLINE_abstracts',
  'knowledge_source_type': 'publications',
  'version': '1970-01_to_2019-12'}]

### For HPO

In [8]:
disease_gene3_disgenet_hpo = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-HPO_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-phenotype"],
        x_bte_reusable_info["provenance"]["disgenet-HPO_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [9]:
disease_gene3_disgenet_hpo["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_phenotype',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'HPO_annotations', 'knowledge_source_type': 'knowledgebase'}]

### For UNIPROT

In [10]:
disease_gene4_disgenet_uniprot = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-UNIPROT_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-protein-var"],
        x_bte_reusable_info["provenance"]["disgenet-UNIPROT_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [11]:
disease_gene4_disgenet_uniprot["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_protein_variant',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'UniProt',
  'knowledge_source_type': 'knowledgebase',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:9606']}}]

### For CGI

In [12]:
disease_gene5_disgenet_cgi = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CGI_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-CGI_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [13]:
disease_gene5_disgenet_cgi["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'Cancer_Genome_Interpreter',
  'knowledge_source_type': 'knowledgebase'}]

### For CLINVAR

In [14]:
disease_gene6_disgenet_clinvar = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CLINVAR_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-seq-var"],
        x_bte_reusable_info["provenance"]["disgenet-CLINVAR_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [15]:
disease_gene6_disgenet_clinvar["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_sequence_variant',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'ClinVar', 'knowledge_source_type': 'knowledgebase'}]

### For GWASCAT

In [16]:
disease_gene7_disgenet_gwascat = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-GWASCAT_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-seq-var"],
        x_bte_reusable_info["provenance"]["disgenet-GWASCAT_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [17]:
disease_gene7_disgenet_gwascat["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_sequence_variant',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'NHGRI_EBI_GWAS_CATALOG', 'knowledge_source_type': 'knowledgebase'}]

### For GWASDB

In [18]:
disease_gene8_disgenet_gwasdb = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-GWASDB_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-seq-var"],
        x_bte_reusable_info["provenance"]["disgenet-GWASDB_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [19]:
disease_gene8_disgenet_gwasdb["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_sequence_variant',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'GWASdb', 'knowledge_source_type': 'knowledgebase'}]

### For MGD

In [20]:
disease_gene9_disgenet_mgd = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-MGD_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-orthology"],
        x_bte_reusable_info["provenance"]["disgenet-MGD_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [21]:
disease_gene9_disgenet_mgd["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_orthology',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'MGD',
  'knowledge_source_type': 'knowledgebase',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:10090']}}]

### For RGD

In [22]:
disease_gene10_disgenet_rgd = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-RGD_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-orthology"],
        x_bte_reusable_info["provenance"]["disgenet-RGD_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [23]:
disease_gene10_disgenet_rgd["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_orthology',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'RGD', 'knowledge_source_type': 'knowledgebase'}]

### For CTD_mouse

In [24]:
disease_gene11_disgenet_ctdmouse = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CTDmouse_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-orthology"],
        x_bte_reusable_info["provenance"]["disgenet-CTDmouse_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [25]:
disease_gene11_disgenet_ctdmouse["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_orthology',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'CTD',
  'knowledge_source_type': 'knowledgebase',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:10090']}}]

### For CTD_rat

In [26]:
disease_gene12_disgenet_ctdrat = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CTDrat_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-propagate-orthology"],
        x_bte_reusable_info["provenance"]["disgenet-CTDrat_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [27]:
disease_gene12_disgenet_ctdrat["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'propagate_from_orthology',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'CTD',
  'knowledge_source_type': 'knowledgebase',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:10116']}}]

### For CLINGEN

In [28]:
disease_gene13_disgenet_clingen = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CLINGEN_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-CLINGEN_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [29]:
disease_gene13_disgenet_clingen["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'ClinGen', 'knowledge_source_type': 'knowledgebase'}]

### For GENOMICS_ENGLAND

In [30]:
disease_gene14_disgenet_genomics_england = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-GENOMICS-ENG_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-GENOMICS-ENG_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [31]:
disease_gene14_disgenet_genomics_england["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'Genomics_England_PanelApp',
  'knowledge_source_type': 'knowledgebase'}]

### For CTD_human

In [32]:
disease_gene15_disgenet_ctdhuman = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-CTDhuman_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-CTDhuman_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [33]:
disease_gene15_disgenet_ctdhuman["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'CTD',
  'knowledge_source_type': 'knowledgebase',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:9606']}}]

### For PSYGENET

In [34]:
disease_gene16_disgenet_psygenet = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-PSYGENET_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-PSYGENET_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [35]:
disease_gene16_disgenet_psygenet["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'PsyGeNET', 'knowledge_source_type': 'knowledgebase'}]

### For ORPHANET

In [36]:
disease_gene17_disgenet_orphanet = {
    "supportBatch":True,
    "inputSeparator":",",
    "parameters": {"fields": "disgenet.genes_related_to_disease"},
    "requestBody": x_bte_reusable_info["request"]["disgenet-ORPHANET_input-disease"],
    "inputs": [
        x_bte_reusable_info["input-disease_UMLS"]
    ],
    "outputs": [
        x_bte_reusable_info["output-gene_disgenet_NCBIGene"]
    ],
    "predicate": x_bte_reusable_info["predicate_is-associated_with"],
    "nodes_conflated": x_bte_reusable_info["disgenet_dggd"]["nodes_conflated"],
    "provenance": [
        x_bte_reusable_info["provenance"]["mydisease-method-ingest-consolidate"],
        x_bte_reusable_info["provenance"]["disgenet-method-ingest-only"],
        x_bte_reusable_info["provenance"]["disgenet-ORPHANET_origin"]        
    ],
    "publications": {"pmid": sample_GDA_response["pubmed"]},
    "numeric_measures_present":True,
    "numeric_measures": x_bte_reusable_info["disgenet_dggd"]["numeric_measures"],
    "category_measures_present":False
    }

In [37]:
disease_gene17_disgenet_orphanet["provenance"]

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'ingest',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'Orphanet', 'knowledge_source_type': 'knowledgebase'}]